# DVC Options - Metrics and Plots
## The DVC way

In the following part we will look into metrics and plots from ZnTrack Nodes.
All `dvc run` options listed [here](https://dvc.org/doc/command-reference/run#options) can be used via `dvc.<option>`.
With the exception of params, which is handled automatically.
All these options take either `str` or `pathlib.Path` directed to the file the content should be stored in.
As shown before, `dvc.deps` can also take another `Node` as an argument.

In [1]:
from zntrack import Node, dvc, zn, config
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
config.nb_name = "04_metrics_and_plots.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmp3zeef6gq/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In the following we define a simple Node that produces a metric and a plot output using `json` and `pandas`.
We will queue multiple experiments with different outputs and then compare them afterwards.

In [5]:
class MetricAndPlot(Node):
    my_metric: Path = dvc.metrics(Path("my_metric.json"))
    my_plots: Path = dvc.plots("my_plots.csv")
    pre_factor = zn.params()

    def __init__(self, pre_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.pre_factor = pre_factor

    def run(self):
        self.my_metric.write_text(
            json.dumps(
                {"metric_1": 17 * self.pre_factor, "metric_2": 42 * self.pre_factor}
            )
        )

        x_data = np.linspace(0, 1.0 * self.pre_factor, 1000)
        y_data = np.exp(x_data)
        df = pd.DataFrame({"y": y_data, "x": x_data}).set_index("x")

        df.to_csv(self.my_plots)

In [6]:
MetricAndPlot().write_graph()
!dvc repro
!git add .
!git commit -m "First Run"

2022-02-22 13:47:44,103 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-22 13:47:47,824 (WARNING): Running DVC command: 'dvc run -n MetricAndPlot ...'
ore>Running stage 'MetricAndPlot':
> python3 -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.
[master (root-commit) 86351ef] First Run
 11 files changed, 510 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
 create mode 100644 .gitignore
 create mode 100644 04_metrics_and_plots.ipynb
 create mode 100644 dvc.lock
 

In [7]:
MetricAndPlot(pre_factor=2).write_graph()
!dvc exp run --queue --name "factor_2"
MetricAndPlot(pre_factor=3).write_graph()
!dvc exp run --queue --name "factor_3"
MetricAndPlot(pre_factor=4).write_graph()
!dvc exp run --queue --name "factor_4"
MetricAndPlot(pre_factor=5).write_graph()
!dvc exp run --queue --name "factor_5"

2022-02-22 13:47:57,919 (WARNING): Running DVC command: 'dvc run -n MetricAndPlot ...'
ore>Queued experiment '5808f4b' for future execution.
2022-02-22 13:48:06,309 (WARNING): Running DVC command: 'dvc run -n MetricAndPlot ...'
ore>Queued experiment '540e21f' for future execution.
2022-02-22 13:48:13,981 (WARNING): Running DVC command: 'dvc run -n MetricAndPlot ...'
ore>Queued experiment '78b4e54' for future execution.
2022-02-22 13:48:21,532 (WARNING): Running DVC command: 'dvc run -n MetricAndPlot ...'
ore>Queued experiment '91a3e82' for future execution.


In [8]:
!dvc exp run --run-all -j 4 > \dev\null

  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
!
  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
  0%|          |.AF9oEGpYHFwttbcNWrE6am.tmp     0.00/4.00 [00:00<?,        ?B/s]
!

  0%|          |.DPe7WKb5LC6n896stx8mCw.tmp        0.00/? [00:00<?,        ?B/s]
! 0%|          |.AF9oEGpYHFwttbcNWrE6am.tmp     0.00/4.00 [00:00<?,        ?B/s]
  0%|          |.DPe7WKb5LC6n896stx8mCw.tmp     0.00/4.00 [00:00<?,        ?B/s]
                                                                                
                                                                                
!

!
  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76     0.00/? [00:00<?,        ?B/s]

  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76  0.00/36.0 [00:00<?,        ?B/s]

  0%|          |4d8533a8bd0c3e8c73e09a05b5bf76  0.00/36.0 [00:00<?,        ?B/s]
                                                                                
                

Now that all experiments are done, we can look at the metrics directly with `dvc exp show` or `dvc metrics show/diff`

In [9]:
!dvc exp show --csv > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

ore>

,rev,typ,Created,parent,metric_1,metric_2,MetricAndPlot.pre_factor,src/MetricAndPlot.py
Experiment,,,,,,,,
NaN,workspace,baseline,NaN,NaN,17.0,42.0,5.0,e743ca7
master,86351ef,baseline,2022-02-22T13:47:53,NaN,17.0,42.0,1.0,e743ca7
factor_3,6417351,branch_commit,2022-02-22T13:48:31,NaN,51.0,126.0,3.0,e743ca7
factor_2,3a09fa0,branch_commit,2022-02-22T13:48:31,NaN,34.0,84.0,2.0,e743ca7
factor_4,2030324,branch_commit,2022-02-22T13:48:31,NaN,68.0,168.0,4.0,e743ca7
factor_5,f98f83e,branch_base,2022-02-22T13:48:31,NaN,85.0,210.0,5.0,e743ca7


We can also use `dvc plots show/diff` to evaluate the plot data that we produced.

In [10]:
!dvc plots diff HEAD factor_2 factor_3 factor_4 factor_5

file:///tmp/tmp3zeef6gq/dvc_plots/index.html


## The ZnTrack way

ZnTrack provides and easier way to handle metrics. Similar to `zn.outs()` which does not require defining a path to outs file, one can use `zn.metrics`.
The same is possible for plots via `zn.plots()` which requires a `pd.DataFrame` with a defined index name.

In [11]:
class ZnTrackMetric(Node):
    my_metric = zn.metrics()
    my_plot = zn.plots()

    def run(self):
        self.my_metric = {"alpha": 1.0, "beta": 0.00473}
        self.my_plot = pd.DataFrame({"val": np.sin(np.linspace(0, 3.14, 100))})
        self.my_plot.index.name = (  # For DVC it is required that the index has a column name
            "index"
        )


ZnTrackMetric().write_graph(no_exec=False)

2022-02-22 13:48:38,977 (WARNING): Running DVC command: 'dvc run -n ZnTrackMetric ...'


In [12]:
!dvc exp show --csv > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

ore>

,rev,typ,Created,parent,metric_1,metric_2,my_metric.alpha,my_metric.beta,MetricAndPlot.pre_factor,src/MetricAndPlot.py,src/ZnTrackMetric.py
Experiment,,,,,,,,,,,
NaN,workspace,baseline,NaN,NaN,17.0,42.0,1.0,0.00473,5.0,e743ca7,e743ca7
master,86351ef,baseline,2022-02-22T13:47:53,NaN,17.0,42.0,NaN,NaN,1.0,e743ca7,NaN
factor_5,f98f83e,branch_commit,2022-02-22T13:48:31,NaN,85.0,210.0,NaN,NaN,5.0,e743ca7,NaN
factor_4,2030324,branch_commit,2022-02-22T13:48:31,NaN,68.0,168.0,NaN,NaN,4.0,e743ca7,NaN
factor_2,3a09fa0,branch_commit,2022-02-22T13:48:31,NaN,34.0,84.0,NaN,NaN,2.0,e743ca7,NaN
factor_3,6417351,branch_base,2022-02-22T13:48:31,NaN,51.0,126.0,NaN,NaN,3.0,e743ca7,NaN


In [13]:
!dvc plots show

file:///tmp/tmp3zeef6gq/dvc_plots/index.html


In [14]:
temp_dir.cleanup()